In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
from skimage import io, transform

train_data = pd.read_csv('../input/chinese-mnist/chinese_mnist.csv')

In [ ]:
# a helper function
def create_file_name(x):
    
    file_name = f"input_{x[0]}_{x[1]}_{x[2]}.jpg"
    return file_name

In [ ]:
train_data["file_name"] = train_data.apply(create_file_name, axis=1)
train_data.head()

In [ ]:
for i in range(15000):
    if train_data['value'][i] == 100:
        train_data['value'][i] = 11
    elif train_data['value'][i] == 1000:
        train_data['value'][i] = 12
    elif train_data['value'][i] == 10000:
        train_data['value'][i] = 13
    elif train_data['value'][i] == 100000000:
        train_data['value'][i] = 14

In [ ]:
from sklearn.model_selection import train_test_split

new_train, new_test = train_test_split(
     train_data, test_size=0.2, random_state=42)

In [ ]:
print(new_train)
print(new_test)

In [ ]:
new_train.to_csv('new_train_file.csv', index=False)
new_test.to_csv('new_test_file.csv', index=False)

In [ ]:
class ChineseMnistDataset(torch.utils.data.Dataset):
    """Dataset wrapping images and target labels for Chinese_mnist

    Arguments:
        A CSV file path
        Path to image folder
        Optional transform to be applied on a sample.
    """
    def __init__(self, csv_file, root_dir, transform=None):
        
        self.label_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.label_frame)
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
            
        # image    
#         print("I am inside the getitem function")
        img_name = os.path.join(self.root_dir, self.label_frame['file_name'].loc[index])
        image = io.imread(img_name)
#         print(image)
        
        # label
        label = self.label_frame['value'].loc[index]
#         print(label)

        sample = (image, label)
        if self.transform:
            sample = (self.transform(image), torch.tensor(label))
            
        return sample

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))])
trainset = ChineseMnistDataset(csv_file='new_train_file.csv',
                                    root_dir='../input/chinese-mnist/data/data/', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
len(trainset)

In [ ]:
len(trainloader)

In [ ]:
%time iter(trainset)

In [ ]:
iter(trainloader)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# The function to show an image.
def imshow(img):
    img = img / 2 + 0.5     # Unnormalize.
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random training images.
dataiter = iter(trainloader)
for i in range(2):
    images, labels = next(dataiter)
    
    # Show images.
    imshow(torchvision.utils.make_grid(images))
    # Print labels.
    print(labels)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 20, 3, padding=1)
        self.fc1 = nn.Linear(5120, 100)
        self.fc2 = nn.Linear(100, 15)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
net = Net()     # Create the network instance.
net.to(device)  # Move the network parameters to the specified device.

In [ ]:
# We use cross-entropy as loss function.
loss_func = nn.CrossEntropyLoss()  
# We use stochastic gradient descent (SGD) as optimizer.
opt = optim.SGD(net.parameters(), lr=0.01, momentum=0.9) 

In [ ]:
avg_losses = []   # Avg. losses.
epochs = 5       # Total epochs.
print_freq = 100  # Print frequency.

for epoch in range(epochs):  # Loop over the dataset multiple times.
    
    running_loss = 0.0       # Initialize running loss.
    

    for i, data in enumerate(trainloader, 0):
        
        # Get the inputs.
        inputs, labels = data
        
        # Move the inputs to the specified device.
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients.
        opt.zero_grad()

        # Forward step.
#         inputs = inputs.unsqueeze(0)
        outputs = net(inputs)
#         labels = labels.view(1)
        loss = loss_func(outputs, labels)
        
        # Backward step.
        loss.backward()
        
        # Optimization step (update the parameters).
        opt.step()

        # Print statistics.
        running_loss += loss.item()
        if i % print_freq == print_freq - 1: # Print every several mini-batches.
            avg_loss = running_loss / print_freq
            print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}'.format(
                epoch, i, avg_loss))
            avg_losses.append(avg_loss)
            running_loss = 0.0



            
            
print('Finished Training.')

In [ ]:
plt.plot(avg_losses)
plt.xlabel('mini-batch index / {}'.format(print_freq))
plt.ylabel('avg. mini-batch loss')
plt.show()

In [ ]:
testset = ChineseMnistDataset(csv_file='new_test_file.csv',
                                    root_dir='../input/chinese-mnist/data/data/', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=True, num_workers=2)

# Check several images.

dataiter = iter(testloader)
images, labels = next(dataiter)
# Show images.
imshow(torchvision.utils.make_grid(images))

print(labels)

outputs = net(images.to(device))
_, predicted = torch.max(outputs, 1)

print(predicted)

In [ ]:
# Get test accuracy.
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))